In [2]:
from utils.df_handle import *

Default bq project:  spatial-vision-343005.biteam


In [ ]:
# from utils.df_handle import *
import pendulum
from airflow import DAG
from airflow.operators.dummy_operator import DummyOperator
from airflow.operators.python_operator import PythonOperator

local_tz = pendulum.timezone("Asia/Bangkok")

name='IBD'
prefix='SYNC_'
csv_path = '/usr/local/airflow/plugins'+'/'

dag_params = {
    'owner': 'airflow',
    "depends_on_past": False,
    'start_date': datetime(2022, 5, 10, tzinfo=local_tz),
    'email_on_failure': True,
    'email_on_retry': False,
    'email':['duyvq@merapgroup.com', 'vanquangduy10@gmail.com'],
    'do_xcom_push': False,
    'execution_timeout':timedelta(seconds=300)
    # 'retries': 3,
    # 'retry_delay': timedelta(minutes=10),
}

dag = DAG(prefix+name,
          catchup=False,
          default_args=dag_params,
          schedule_interval= '*/30 8-17,23-23 * * *',
          tags=[prefix+name, 'Sync', '30mins']
)

In [3]:
start_date = '2022-01-01'
datenow = datetime.now().strftime("%Y-%m-%d")
datenow_mns1 = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")
fdom = datetime.now().replace(day=1).strftime("%Y-%m-%d")

In [4]:
from_tb = "OM_IssueBookDet"

In [5]:
table_name = "sync_dms_ibd"
table_temp = "sync_dms_ibd_temp"

In [8]:
sql = \
f"""
DECLARE @from DATE = '{start_date}'
DECLARE @to DATE = '2022-01-31'
SELECT
CONCAT(BranchID,BatNbr,OrderNbr) as pk,
BranchID,
BatNbr,
OrderNbr,
Status,
DeliveryTime,
Crtd_DateTime,
Crtd_Prog,
Crtd_User,
LUpd_DateTime,
Transporters
from {from_tb}
where Crtd_Datetime >= @from
"""

In [7]:
df = get_ms_df(sql)
df['inserted_at'] = datetime.now()
bq_values_insert(df, f"{table_temp}", 3)
sql = \
f"""
DROP TABLE IF EXISTS biteam.{table_name};
CREATE TABLE biteam.{table_name} LIKE biteam.{table_temp}
PARTITION BY DATE(crtd_datetime)
CLUSTER BY pk,branchid,batnbr,ordernbr
"""
execute_bq_query(sql)

1it [00:16, 16.83s/it]


In [9]:
df = get_ms_df(sql)
df['inserted_at'] = datetime.now()
bq_values_insert(df, f"{table_name}", 2)

1it [00:13, 13.59s/it]


In [9]:
usql = \
f"""
DECLARE @from DATE = '{datenow}'
DECLARE @to DATE = '2022-01-31'
SELECT
BranchID,
BatNbr,
OrderNbr,
Status,
DeliveryTime,
Crtd_DateTime,
Crtd_Prog,
Crtd_User,
LUpd_DateTime,
Transporters
from {from_tb}
where LUpd_DateTime >= @from
and Crtd_DateTime < @from
"""

In [10]:
table_name = "sync_dms_ibd"
table_temp = "sync_dms_ibd_temp"

In [6]:
#INSERT
df = get_ms_df(sql)
df['inserted_at'] = datetime.now()
# df['inserted_At']

# bq_values_insert(df, f"{table_temp}", 3)
# sql = \
# f"""
# DROP TABLE IF EXISTS biteam.{table_name};
# CREATE TABLE biteam.{table_name} LIKE biteam.{table_temp}
# PARTITION BY DATE(crtd_datetime)
# CLUSTER BY branchid,batnbr,ordernbr
# """
# print(sql)
# execute_bq_query(sql)

dsql = \
f"""
delete from biteam.{table_name} where date(crtd_datetime) >= '{datenow}'
"""
print("delete_sql: ", dsql)
execute_bq_query(dsql)

bq_values_insert(df, f"{table_name}", 2)
# sql = \
# f"""
# DROP TABLE IF EXISTS biteam.{table_temp};
# """
# execute_bq_query(sql)

In [13]:
#UPDATE
df_update = get_ms_df(usql)
df_update.columns = lower_col(df_update)
# df_update.columns
df_update['crtd_datetime'] = df_update.crtd_datetime.dt.normalize()
df_update1 = df_update['crtd_datetime']
df_update1.drop_duplicates(inplace=True)
tpl_dt = tuple(df_update1.dt.strftime('%Y-%m-%d').to_list()) + ('1900-01-01',)
# tpl_dt
df_update1 = df_update['branchid'] + df_update['batnbr'] + df_update['ordernbr']
df_update1.drop_duplicates(inplace=True)
tpl_pk = tuple(df_update1.to_list()) + ('',)
del_sql = \
f"""
DELETE FROM biteam.{table_name}
WHERE
DATE(crtd_datetime) in {tpl_dt}
AND concat(branchid,batnbr,ordernbr) in {tpl_pk}
"""
print("del_sql ",del_sql)
execute_bq_query(del_sql)
df_update['inserted_at'] = datetime.now()
bq_values_insert(df_update, f"{table_name}", 2)

del_sql  
DELETE FROM biteam.sync_dms_ibd
WHERE
DATE(crtd_datetime) in ('2022-05-13', '2022-05-12')
AND concat(branchid,batnbr,ordernbr) in ('MR0001202205000001039DH0-0522-00932', 'MR0001202205000001039DH0-0522-00935', 'MR0001202205000001039DH0-0522-00949', 'MR0001202205000001039DH0-0522-02122', 'MR0001202205000001039DH0-0522-02124', 'MR0001202205000001041DH0-0522-01988', 'MR0001202205000001041DH0-0522-01991', 'MR0001202205000001041DH0-0522-02007', 'MR0001202205000001041DH0-0522-02137', 'MR0001202205000001041DH0-0522-02151', 'MR0001202205000001042DH0-0522-01982', 'MR0001202205000001042DH0-0522-01985', 'MR0001202205000001042DH0-0522-02057', 'MR0001202205000001042DH0-0522-02107', 'MR0001202205000001042DH0-0522-02128', 'MR0001202205000001042DH0-0522-02146', 'MR0001202205000001042EP120522-00250', 'MR0001202205000001043DH0-0522-01083', 'MR0001202205000001043DH0-0522-01084', 'MR0001202205000001043DH0-0522-01238', 'MR0001202205000001043DH0-0522-01909', 'MR0001202205000001043DH0-0522-02132', '

In [11]:
print(usql)


DECLARE @from DATE = '2022-05-14'
DECLARE @to DATE = '2022-01-31'
SELECT
BranchID,
BatNbr,
OrderNbr,
Status,
DeliveryTime,
Crtd_DateTime,
Crtd_Prog,
Crtd_User,
LUpd_DateTime,
Transporters
from OM_IssueBookDet
where LUpd_DateTime >= @from
and Crtd_DateTime < @from

